In [1]:
# =============================================================================
# IFAE (Income-Health-Access Equity) ZIP/ZCTA Ranking — with AQI + HHI
# - Composite (percentiles) + IsolationForest anomaly
# - Population-based access (people per pharmacy)
# - Auto-disable low-coverage features; impute income/health by national medians
# - No state-relative, no population/density impact weighting
#
# Outputs:
#   results/national_ifae_rank.csv   (full ranking)
#   results/top5_ifae.csv            (top K with population >= 1,000)
#   results/bottom5_ifae.csv         (bottom K)
# =============================================================================

import os, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

# === CONFIG: set your file paths here ===
FINANCIAL_CSV  = "data/financial_data.csv"      # needs: NAME, S1901_C01_012E (median HH income)
HEALTH_CSV     = "data/health_data.csv"         # needs: ZCTA5, GHLTH_CrudePrev
PHARMACY_CSV   = "data/pharmacy_data.csv"       # needs: ZIP, NAME, X, Y (X/Y optional)
POPULATION_CSV = "data/population_data.csv"     # teammate used skiprows=10; autodetect columns

# Optional extras (will be auto-disabled if missing/low coverage)
AQI_CSV        = "data/AQI_data.csv"            # PM2.5; must include ZIP, Arithmetic Mean, Observation Count
HHI_XLSX       = "data/HHI_data.xlsx"           # Excel; must include ZCTA, HHB_SCORE (heat)

OUT_DIR        = "results"
CONTAMINATION  = 0.03
TOP_K          = 10
MIN_POP_TOPK   = 1000   # population gate for top-k presentation

# ---------- Monitoring helpers ----------
_T0 = time.time()
def stamp(msg):
    now = datetime.now().strftime("%H:%M:%S")
    print(f"[{now} +{time.time()-_T0:6.2f}s] {msg}", flush=True)

class Step:
    def __init__(self, name): self.name=name; self.t0=None
    def __enter__(self): self.t0=time.time(); stamp(f"▶ {self.name} ..."); return self
    def __exit__(self, et, ev, tb):
        dt = time.time()-self.t0
        stamp(("✓ " if et is None else "✖ ") + f"{self.name} {'done' if et is None else 'failed'} in {dt:.2f}s")

def read_csv_smart(path, **kw):
    p = Path(path)
    if not p.exists(): raise FileNotFoundError(f"Missing file: {path}")
    try:
        return pd.read_csv(path, low_memory=False, **kw)
    except Exception as e:
        stamp(f"CSV read warning: {e}. Retrying with simpler defaults.")
        return pd.read_csv(path)

def coerce_zcta(series):
    """Return 5-digit, zero-padded strings; strips non-digit chars if present."""
    s = series.astype(str).str.extract(r"(\d{3,5})", expand=False)
    return s.fillna("").str.zfill(5)

def pct_rank(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors='coerce')
    return s.rank(pct=True, method='average').fillna(0.5)

def nonint_fraction(x: pd.Series) -> float:
    x = pd.to_numeric(x, errors='coerce')
    return float((np.abs(x - np.round(x)) > 1e-9).mean())

# ---------- Optional loaders: AQI + HHI ----------
def read_aqi_data(file_path):
    """
    Reads AQI/PM2.5-like CSV and aggregates annual weighted average per ZIP by Observation Count.
    Returns: aqi_annual: [zip, aqi, obs_total]
    """
    df = read_csv_smart(file_path)
    zip_cols = [c for c in df.columns if str(c).upper().startswith('ZIP')]
    if not zip_cols:
        raise KeyError("AQI CSV needs a ZIP column.")
    zip_col = zip_cols[-1]
    df['zip'] = df[zip_col].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    if 'Arithmetic Mean' not in df.columns or 'Observation Count' not in df.columns:
        raise KeyError("AQI CSV must include 'Arithmetic Mean' and 'Observation Count' columns.")
    val_col = 'Arithmetic Mean'
    w_col   = 'Observation Count'
    df[val_col] = pd.to_numeric(df[val_col], errors='coerce')
    df[w_col]   = pd.to_numeric(df[w_col], errors='coerce').fillna(0)

    # Restrict to PM2.5 if present
    if 'Parameter Name' in df.columns:
        df = df[df['Parameter Name'].astype(str).str.contains('PM2.5', na=False)]

    df = df.dropna(subset=['zip', val_col])
    df = df[df[w_col] > 0]

    grp_annual = df.groupby('zip', as_index=False).apply(
        lambda g: pd.Series({
            'aqi': np.average(g[val_col], weights=g[w_col]),
            'obs_total': g[w_col].sum()
        })
    ).reset_index(drop=True)

    return grp_annual[['zip','aqi','obs_total']]

def read_hhi_excel(file_path):
    """Reads Heat-Health Index Excel -> [zip, heat_hhb, nbe_score?, hhi_overall?]"""
    df = pd.read_excel(file_path, dtype={'ZCTA': str})
    if 'ZCTA' not in df.columns:
        raise ValueError("HHI Excel must contain 'ZCTA' column.")
    df['zip'] = df['ZCTA'].astype(str).str.extract(r'(\d{5})')[0].fillna('').str.zfill(5)

    out = pd.DataFrame({'zip': df['zip']})
    if 'HHB_SCORE' in df.columns:
        out['heat_hhb'] = pd.to_numeric(df['HHB_SCORE'], errors='coerce')
    if 'NBE_SCORE' in df.columns:
        out['nbe_score'] = pd.to_numeric(df['NBE_SCORE'], errors='coerce')
    if 'OVERALL_SCORE' in df.columns:
        out['hhi_overall'] = pd.to_numeric(df['OVERALL_SCORE'], errors='coerce')

    return out.dropna(subset=['zip']).drop_duplicates(subset=['zip'])

# ---------- Load & prepare each source ----------
with Step("Load FINANCIAL (income)"):
    fin = read_csv_smart(FINANCIAL_CSV)
    zcta_col = None
    for cand in ["ZCTA5","zcta5","ZCTA","zcta","ZIP","Zip","NAME","Name","name"]:
        if cand in fin.columns:
            zcta_col = cand; break
    if zcta_col is None:
        raise KeyError(f"Could not find a ZCTA/ZIP column in {FINANCIAL_CSV}.")
    if "S1901_C01_012E" not in fin.columns:
        raise KeyError("Missing S1901_C01_012E (median HH income) in FINANCIAL_CSV.")
    fin = fin.rename(columns={zcta_col:"ZCTA5"})
    fin["ZCTA5"] = coerce_zcta(fin["ZCTA5"])
    fin = fin[["ZCTA5","S1901_C01_012E"]].copy()
    fin["S1901_C01_012E"] = pd.to_numeric(fin["S1901_C01_012E"], errors="coerce")
    stamp(f"FIN rows={len(fin)}, null income={fin['S1901_C01_012E'].isna().sum()}")

with Step("Load HEALTH (poor general health %)"):
    hlth = read_csv_smart(HEALTH_CSV)
    if "ZCTA5" not in hlth.columns:
        for cand in ["ZCTA","zcta","ZIP","Zip","name","NAME"]:
            if cand in hlth.columns:
                hlth = hlth.rename(columns={cand:"ZCTA5"})
                break
    if "ZCTA5" not in hlth.columns or "GHLTH_CrudePrev" not in hlth.columns:
        raise KeyError("HEALTH_CSV must contain ZCTA5 and GHLTH_CrudePrev.")
    hlth["ZCTA5"] = coerce_zcta(hlth["ZCTA5"])
    hlth["GHLTH_CrudePrev"] = pd.to_numeric(hlth["GHLTH_CrudePrev"], errors="coerce")
    hlth = hlth[["ZCTA5","GHLTH_CrudePrev"]]
    stamp(f"HEALTH rows={len(hlth)}, null health%={hlth['GHLTH_CrudePrev'].isna().sum()}")

with Step("Load PHARMACY (access)"):
    ph = read_csv_smart(PHARMACY_CSV)
    for needed in ["ZIP","NAME"]:
        if needed not in ph.columns:
            raise KeyError(f"PHARMACY_CSV must contain {needed}.")
    ph["ZCTA5"] = coerce_zcta(ph["ZIP"])
    ph_cnt = ph.groupby("ZCTA5", dropna=False)["NAME"].nunique(dropna=True).reset_index()
    ph_cnt = ph_cnt.rename(columns={"NAME":"pharmacies_count"})
    stamp(f"PHARMACY unique ZCTA5={ph_cnt['ZCTA5'].nunique()}, pharmacies={ph_cnt['pharmacies_count'].sum()}")

with Step("Load POPULATION (skiprows=10 + autodetect)"):
    pop = read_csv_smart(POPULATION_CSV, skiprows=10)
    code_col = None
    for cand in ["ZCTA5","ZCTA","ZIP","Zip","GEOID","geoid","NAME","name"]:
        if cand in pop.columns:
            code_col=cand; break
    if code_col is None:
        obj_cols = [c for c in pop.columns if pop[c].dtype == object]
        code_col = obj_cols[0] if obj_cols else pop.columns[0]
    pop = pop.rename(columns={code_col:"ZCTA5"})
    pop["ZCTA5"] = coerce_zcta(pop["ZCTA5"])

    pop_col = None
    for cand in ["POP","Population","population","TOTAL_POP","TotPop","DP05_0001E","pop"]:
        if cand in pop.columns:
            pop_col=cand; break
    if pop_col is None:
        num_cols = [c for c in pop.columns if pd.api.types.is_numeric_dtype(pop[c])]
        sums = {c: pd.to_numeric(pop[c], errors="coerce").sum(skipna=True) for c in num_cols}
        pop_col = max(sums, key=sums.get) if sums else None
    if pop_col is None:
        raise KeyError("Could not infer population column in POPULATION_CSV.")

    land_col = None
    for cand in ["land_area_km2","Land_Area_km2","ALAND_KM2","aland_km2","ALAND","area","AREA_KM2","ALAND_SQMI","AREALAND"]:
        if cand in pop.columns:
            land_col = cand; break

    keep_cols = ["ZCTA5", pop_col] + ([land_col] if land_col else [])
    pop = pop[keep_cols].copy()
    pop = pop.rename(columns={pop_col:"population"})
    pop["population"] = pd.to_numeric(pop["population"], errors="coerce")
    if land_col:
        pop = pop.rename(columns={land_col:"land_area_raw"})
        pop["land_area_raw"] = pd.to_numeric(pop["land_area_raw"], errors="coerce")
    stamp(f"POP rows={len(pop)}, null pop={pop['population'].isna().sum()}, land_col={land_col}")

# Optional: load AQI + HHI
try:
    with Step("Load AQI (annual weighted PM2.5)"):
        aqi_annual = read_aqi_data(AQI_CSV)
        stamp(f"AQI rows={len(aqi_annual)} (coverage ~{100*len(aqi_annual)/max(1, pop['ZCTA5'].nunique()):.1f}%)")
except Exception as e:
    aqi_annual = None
    warnings.warn(f"AQI not loaded: {e}")

try:
    with Step("Load HHI (heat vulnerability)"):
        hhi = read_hhi_excel(HHI_XLSX)
        stamp(f"HHI rows={len(hhi)}")
except Exception as e:
    hhi = None
    warnings.warn(f"HHI not loaded: {e}")

# ---------- Merge all to ZCTA frame ----------
with Step("Merge all features by ZCTA/ZIP"):
    df = fin.merge(hlth, on="ZCTA5", how="outer") \
            .merge(pop, on="ZCTA5", how="outer") \
            .merge(ph_cnt, on="ZCTA5", how="left")
    if aqi_annual is not None and not aqi_annual.empty:
        df = df.merge(aqi_annual.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    if hhi is not None and not hhi.empty:
        df = df.merge(hhi.rename(columns={'zip':'ZCTA5'}), on="ZCTA5", how="left")
    stamp(f"Merged shape: {df.shape}")

# ---------- Feature engineering ----------
with Step("Feature engineering"):
    # Rename to friendly
    df["median_income"]   = pd.to_numeric(df["S1901_C01_012E"], errors="coerce")
    df["poor_health_pct"] = pd.to_numeric(df["GHLTH_CrudePrev"], errors="coerce")
    df["pharmacies_count"]= pd.to_numeric(df["pharmacies_count"], errors="coerce").fillna(0)

    # Impute income & health with national medians
    for col in ["median_income","poor_health_pct"]:
        med = df[col].median(skipna=True)
        df[col] = df[col].fillna(med)

    # Access: population-based scarcity (people per pharmacy; higher = worse)
    denom = df["pharmacies_count"].replace(0, 1)  # keep finite for 0-pharmacy ZIPs
    df["pop_per_pharmacy"] = np.where(df["population"].gt(0), df["population"] / denom, np.nan)

    # Density (optional; convert ALAND sq meters to km2 if present)
    if "land_area_raw" in df.columns:
        med_area = np.nanmedian(df["land_area_raw"])
        if np.isfinite(med_area) and med_area > 1e5:
            df["land_area_km2"] = df["land_area_raw"] / 1e6
        else:
            df["land_area_km2"] = df["land_area_raw"]
        df["pop_density"] = np.where(df["land_area_km2"].gt(0), df["population"]/df["land_area_km2"], np.nan)
    else:
        df["pop_density"] = np.nan

    # AQI/HHI numeric
    if "aqi" in df.columns:       df["aqi"] = pd.to_numeric(df["aqi"], errors="coerce")
    if "heat_hhb" in df.columns:  df["heat_hhb"] = pd.to_numeric(df["heat_hhb"], errors="coerce")

    # Percentile features (0..1, higher = worse)
    df["income_pct_inv"]  = 1 - pct_rank(df["median_income"])
    df["health_pct"]      = pct_rank(df["poor_health_pct"])
    df["access_pct_inv"]  = pct_rank(df["pop_per_pharmacy"])
    df["density_pct"]     = pct_rank(df["pop_density"]) if df["pop_density"].notna().any() else 0.5
    df["aqi_pct"]         = pct_rank(df["aqi"])         if "aqi" in df.columns else 0.5
    df["heat_pct"]        = pct_rank(df["heat_hhb"])    if "heat_hhb" in df.columns else 0.5

# ---------- Composite + IsolationForest ----------
with Step("Compute composite & train IsolationForest"):
    # Coverage checks (fraction of non-null)
    cov = {
        'aqi':    ("aqi" in df.columns) and df["aqi"].notna().mean(),
        'heat':   ("heat_hhb" in df.columns) and df["heat_hhb"].notna().mean(),
        'density': df["pop_density"].notna().mean()
    }
    print("Coverage:", cov)

    use_aqi  = bool(cov["aqi"])  and (cov["aqi"]  >= 0.30)
    use_heat = bool(cov["heat"]) and (cov["heat"] >= 0.30)
    use_dens = bool(cov["density"]) and (cov["density"] >= 0.20)

    # Base weights (sum ~1 before normalization)
    w_income, w_health, w_access = 0.30, 0.35, 0.30
    w_aqi   = 0.05 if use_aqi  else 0.00
    w_heat  = 0.05 if use_heat else 0.00
    w_dens  = 0.00 if not use_dens else 0.02  # keep small even if available

    # Normalize to sum=1
    Wsum = w_income + w_health + w_access + w_aqi + w_heat + w_dens
    w_income, w_health, w_access, w_aqi, w_heat, w_dens = [w/Wsum for w in [w_income, w_health, w_access, w_aqi, w_heat, w_dens]]

    # Composite (transparent)
    df["composite"] = (
        w_income * df["income_pct_inv"] +
        w_health * df["health_pct"] +
        w_access * df["access_pct_inv"] +
        w_aqi    * (df["aqi_pct"]   if use_aqi  else 0) +
        w_heat   * (df["heat_pct"]  if use_heat else 0) +
        w_dens   * (df["density_pct"] if use_dens else 0)
    )

    # IF features = same percentile vector
    feat_cols = ["income_pct_inv","health_pct","access_pct_inv"]
    if use_aqi:  feat_cols.append("aqi_pct")
    if use_heat: feat_cols.append("heat_pct")
    if use_dens: feat_cols.append("density_pct")
    X = df[feat_cols].fillna(0.5).to_numpy()

    iforest = IsolationForest(random_state=42, contamination=CONTAMINATION, n_estimators=150, n_jobs=-1)
    iforest.fit(X)
    anom = -iforest.decision_function(X)                     # invert so higher = more anomalous
    anom = (anom - anom.min()) / (anom.max() - anom.min() + 1e-12)  # 0..1
    df["iforest_anomaly"] = anom

    # Final score = blend
    df["IFAE_score"] = 0.5*df["composite"] + 0.5*df["iforest_anomaly"]

# ---------- Finalize outputs ----------
with Step("Finalize outputs"):
    out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)

    keep = [
        "ZCTA5","IFAE_score","composite","iforest_anomaly",
        "median_income","poor_health_pct","population","pharmacies_count",
        "pop_per_pharmacy","income_pct_inv","health_pct","access_pct_inv",
        "density_pct","pop_density"
    ]
    if "aqi" in df.columns:       keep += ["aqi","aqi_pct","obs_total"]
    if "heat_hhb" in df.columns:  keep += ["heat_hhb","heat_pct"]
    if "land_area_km2" in df.columns: keep += ["land_area_km2"]

    ranked = df[keep].copy().sort_values("IFAE_score", ascending=False).reset_index(drop=True)

    # Save everything
    out_full   = out / "national_ifae_rank.csv"
    out_top    = out / "top5_ifae.csv"
    out_bottom = out / "bottom5_ifae.csv"
    ranked.to_csv(out_full, index=False)

    # Population gate for top-k presentation
    eligible = ranked[ ranked["population"].fillna(0) >= MIN_POP_TOPK ]
    top_presentable = eligible.head(TOP_K)
    bottom_presentable = ranked.tail(TOP_K)

    top_presentable.to_csv(out_top, index=False)
    bottom_presentable.to_csv(out_bottom, index=False)

    stamp(f"Wrote {out_full}")
    stamp(f"Wrote {out_top} (pop ≥ {MIN_POP_TOPK})")
    stamp(f"Wrote {out_bottom}")

    print("\nTop 5 (pop ≥ {:,}) preview:\n".format(MIN_POP_TOPK), top_presentable.head(5).to_string(index=False))
    print("\nBottom 5 preview:\n", bottom_presentable.tail(5).to_string(index=False))

stamp("All done. 🚀")


[11:20:59 +  0.00s] ▶ Load FINANCIAL (income) ...
[11:21:00 +  0.34s] FIN rows=33773, null income=3264
[11:21:00 +  0.34s] ✓ Load FINANCIAL (income) done in 0.34s
[11:21:00 +  0.35s] ▶ Load HEALTH (poor general health %) ...
[11:21:00 +  0.55s] HEALTH rows=32520, null health%=0
[11:21:00 +  0.56s] ✓ Load HEALTH (poor general health %) done in 0.21s
[11:21:00 +  0.56s] ▶ Load PHARMACY (access) ...
[11:21:00 +  1.06s] PHARMACY unique ZCTA5=14940, pharmacies=61970
[11:21:00 +  1.06s] ✓ Load PHARMACY (access) done in 0.50s
[11:21:00 +  1.06s] ▶ Load POPULATION (skiprows=10 + autodetect) ...
[11:21:00 +  1.10s] POP rows=40959, null pop=0, land_col=None
[11:21:00 +  1.10s] ✓ Load POPULATION (skiprows=10 + autodetect) done in 0.05s
[11:21:00 +  1.10s] ▶ Load AQI (annual weighted PM2.5) ...
[11:21:02 +  2.61s] AQI rows=125 (coverage ~0.3%)
[11:21:02 +  2.61s] ✓ Load AQI (annual weighted PM2.5) done in 1.51s
[11:21:02 +  2.61s] ▶ Load HHI (heat vulnerability) ...


/var/folders/9d/5gzqwh3d5sx43n9fsy97czyc0000gn/T/ipykernel_60040/2731130997.py:98: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grp_annual = df.groupby('zip', as_index=False).apply(


[11:21:09 +  9.61s] HHI rows=32092
[11:21:09 +  9.61s] ✓ Load HHI (heat vulnerability) done in 6.99s
[11:21:09 +  9.61s] ▶ Merge all features by ZCTA/ZIP ...
[11:21:09 +  9.65s] Merged shape: (41094, 10)
[11:21:09 +  9.65s] ✓ Merge all features by ZCTA/ZIP done in 0.04s
[11:21:09 +  9.65s] ▶ Feature engineering ...
[11:21:09 +  9.66s] ✓ Feature engineering done in 0.01s
[11:21:09 +  9.66s] ▶ Compute composite & train IsolationForest ...
Coverage: {'aqi': np.float64(0.003041806589769796), 'heat': np.float64(0.7806979121039568), 'density': np.float64(0.0)}
[11:21:09 + 10.10s] ✓ Compute composite & train IsolationForest done in 0.44s
[11:21:09 + 10.10s] ▶ Finalize outputs ...
[11:21:10 + 10.31s] Wrote results/national_ifae_rank.csv
[11:21:10 + 10.31s] Wrote results/top5_ifae.csv (pop ≥ 1000)
[11:21:10 + 10.31s] Wrote results/bottom5_ifae.csv

Top 5 (pop ≥ 1,000) preview:
 ZCTA5  IFAE_score  composite  iforest_anomaly  median_income  poor_health_pct  population  pharmacies_count  pop_per_p